In [1]:
library(brms) #predictability: Double hierearchical model
library(MCMCglmm) #HPDinterval
library(dplyr) #%>%
library(lmerTest) #lmer
#library(MuMIn) #r2
#library(merTools) #sim

Warning message:
"le package 'brms' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Rcpp

Warning message:
"le package 'Rcpp' a été compilé avec la version R 4.1.2"
Loading 'brms' package (version 2.16.3). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attachement du package : 'brms'


L'objet suivant est masqué depuis 'package:stats':

    ar


Warning message:
"le package 'MCMCglmm' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : Matrix

Warning message:
"le package 'Matrix' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : coda

Warning message:
"le package 'coda' a été compilé avec la version R 4.1.2"
Le chargement a nécessité le package : ape

Warning message:
"le package 'ape' a été compilé avec la version R 4.1.2"

Attachement du package : 'MCMCglmm'


L'objet suivant est masqué depuis 'pac

In [2]:
library(parallel)
my.cores = detectCores()

In [3]:
#df[rowSums(is.na(df)) > 0,]

# Behaviorual syndrome

In [4]:
path_ = 'G:/VPHI/Welfare/2- Research Projects/OFHE2.OriginsE2/DataOutput/TrackingSystem/ALLDATA_/chapter2'
#df = read.csv(file.path(path_,'df_daily_aggregatedHA.csv'), header = TRUE, sep = ",")
df = read.csv(file.path(path_,'df_weekly_BT.csv'), header = TRUE, sep = ",")

df$HenID = as.factor(df$HenID)   
df$PenID = as.factor(df$PenID) 

df$CLASS = as.factor(df$CLASS) 
df$Treatment = as.factor(df$Treatment)

#scale: mean-centering of the environmental variable so that intercepts reflect average values for the population and ind. 
#df$temperature_C_avg_scale = scale(df$temperature_C_avg, center=TRUE, scale=TRUE)

#setting reference group
contrasts(df$Treatment)
df <- df %>% mutate(Treatment = relevel(Treatment, ref = "TRAN"))
contrasts(df$Treatment)

contrasts(df$CLASS)
df <- df %>% mutate(CLASS = relevel(CLASS, ref = "LEXP"))
contrasts(df$CLASS)
print(dim(df))
summary(df)

,TRAN
OFH,0
TRAN,1


,OFH
TRAN,0
OFH,1


,MEXP,NewAfterEpi,REXP
LEXP,0,0,0
MEXP,1,0,0
NewAfterEpi,0,1,0
REXP,0,0,1


,MEXP,NewAfterEpi,REXP
LEXP,0,0,0
MEXP,1,0,0
NewAfterEpi,0,1,0
REXP,0,0,1


[1] 2705   17


     HenID           WOA            PenID             CLASS     Treatment  
 hen_193:  20   Min.   :26.00   pen_10 :372   LEXP       :506   TRAN:1327  
 hen_59 :  20   1st Qu.:33.00   pen_9  :361   MEXP       :620   OFH :1378  
 hen_78 :  20   Median :41.00   pen_5  :358   NewAfterEpi:841              
 hen_112:  19   Mean   :41.65   pen_11 :349   REXP       :738              
 hen_116:  19   3rd Qu.:49.00   pen_8  :339                                
 hen_120:  19   Max.   :59.00   pen_12 :320                                
 (Other):2588                   (Other):606                                
 weekly_avertical_travel_distance weekly_avg_food_related_behavior
 Min.   :  14.0                   Min.   :-0.85432                
 1st Qu.: 322.0                   1st Qu.:-0.21229                
 Median : 431.0                   Median :-0.05761                
 Mean   : 435.5                   Mean   :-0.09527                
 3rd Qu.: 540.0                   3rd Qu.: 0.05294       

# fit DHGLM

In [33]:
mod_dhgm <- bf(
  vertical_travel_distance ~ Treatment + scale(weekly_avg_temperature) + scale(age) +(1|a|HenID),
  sigma ~ Treatment + scale(temperature_C_avg) + scale(age) +(1|a|HenID)
)

mod <- brm(mod_dhgm,
  data = df,
  warmup = 5000, iter = 10000, thin = 2,
  chains = 4, inits = "random",
  seed = 12345,
  cores = my.cores
)

summary(mod)

Compiling Stan program...

Start sampling



 Family: gaussian 
  Links: mu = identity; sigma = log 
Formula: vertical_travel_distance ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID) 
         sigma ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID)
   Data: df (Number of observations: 736) 
  Draws: 4 chains, each with iter = 5000; warmup = 2500; thin = 2;
         total post-warmup draws = 5000

Group-Level Effects: 
~HenID (Number of levels: 207) 
                               Estimate Est.Error l-95% CI u-95% CI Rhat
sd(Intercept)                     18.30      1.07    16.28    20.46 1.00
sd(sigma_Intercept)                0.34      0.05     0.24     0.44 1.00
cor(Intercept,sigma_Intercept)     0.53      0.13     0.25     0.77 1.00
                               Bulk_ESS Tail_ESS
sd(Intercept)                      4499     7029
sd(sigma_Intercept)                4561     7418
cor(Intercept,sigma_Intercept)     5166     6776

Population-Level Effects: 
                             Esti

# fit multivariate model

In [ ]:
['weekly_avertical_travel_distance', 'weekly_avg_food_related_behavior', 'weekly_avg_distribution_entropy',
 'weekly_avg_latency_1_Zone_h', 'weekly_nbr_uncommon_behavior', 'weekly_avg_temperature']

In [47]:
verticalditance_dhgm = bf(scale(weekly_avertical_travel_distance) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID),
                       sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID))

food_dhgm = bf(scale(weekly_avg_food_related_behavior) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID),
                       sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID))

distrentropy_dhgm = bf(scale(weekly_avg_distribution_entropy) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID),
                       sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID))

#latency1_dhgm = bf(scale(weekly_avg_latency_1_Zone_h) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID),
#                       sigma ~ Treatment + scale(temperature_C_avg) + scale(WOA) +(1|a|HenID))

uncommonbehavior_dhgm = bf(scale(weekly_nbr_uncommon_behavior) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID),
                       sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) +(1|a|HenID))


Multi_DHGLM = brm(verticalditance_dhgm + food_dhgm + distrentropy_dhgm + uncommonbehavior_dhgm +
                   set_rescor(FALSE),
                    data = df,
                    iter = 10000, warmup = 6000, thin = 4,
                    chains = 4, cores =my.cores, seed = 12345
                    )

summary(Multi_DHGLM)

Compiling Stan program...

Start sampling

Warning message:
"There were 4000 transitions after warmup that exceeded the maximum treedepth. Increase max_treedepth above 10. See
http://mc-stan.org/misc/warnings.html#maximum-treedepth-exceeded"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is 4.36, indicating chains have not mixed.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#r-hat"
Warning message:
"Bulk Effective Samples Size (ESS) is too low, indicating posterior means and medians may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#bulk-ess"
Warning message:
"Tail Effective Samples Size (ESS) is too low, indicating posterior variances and tail quantiles may be unreliable.
Running the chains for more iterations may help. See
http://mc-stan.org/misc/warnings.html#tail-ess"
Warning message:
"Parts of the model have not con

 Family: MV(gaussian, gaussian, gaussian, gaussian) 
  Links: mu = identity; sigma = log
         mu = identity; sigma = log
         mu = identity; sigma = log
         mu = identity; sigma = log 
Formula: scale(weekly_avertical_travel_distance) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID) 
         sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID)
         scale(weekly_avg_food_related_behavior) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID) 
         sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID)
         scale(weekly_avg_distribution_entropy) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID) 
         sigma ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID)
         scale(weekly_nbr_uncommon_behavior) ~ Treatment + scale(weekly_avg_temperature) + scale(WOA) + (1 | a | HenID) 
         sigma ~ Treatment + scale(we

In [25]:
timeactivity_dhgm = bf(scale(activity_50percentile_h) ~ Treatment + scale(temperature_C_avg) + scale(age) +(1|a|HenID),
                       sigma ~ Treatment + scale(temperature_C_avg) + scale(age) +(1|a|HenID))

verticaldistance_dhgm = bf(scale(vertical_travel_distance) ~ Treatment + scale(temperature_C_avg) + scale(age) +(1|a|HenID),
                           sigma ~ Treatment + scale(temperature_C_avg) + scale(age) +(1|a|HenID))

Multi_DHGLM <- brm(timeactivity_dhgm + verticaldistance_dhgm +
                   set_rescor(FALSE),
                    data = df,
                    iter = 10000, warmup = 6000, thin = 4,
                    chains = 4, cores =my.cores, seed = 12345
                    )

summary(Multi_DHGLM)

Compiling Stan program...

Start sampling



 Family: MV(gaussian, gaussian) 
  Links: mu = identity; sigma = log
         mu = identity; sigma = log 
Formula: scale(activity_50percentile_h) ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID) 
         sigma ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID)
         scale(vertical_travel_distance) ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID) 
         sigma ~ Treatment + scale(temperature_C_avg) + scale(age) + (1 | a | HenID)
   Data: df (Number of observations: 736) 
  Draws: 4 chains, each with iter = 2500; warmup = 1500; thin = 4;
         total post-warmup draws = 1000

Group-Level Effects: 
~HenID (Number of levels: 207) 
                                                                                            Estimate
sd(scaleactivity50percentileh_Intercept)                                                        0.62
sd(sigma_scaleactivity50percentileh_Intercept)                                                